## Functionality that needs to be added

- ~~Trim Raw Input~~
- ~~Night vs Day bouts (critera - choose where over 50% of bout is spent, in case of tie choose night)~~
- ~~Create Sleep Profile~~
- Create summary stats
- Create function to run all flies and arrange in exportable format
- Write to seperate sheets in Excel
- Adapt to python script that can be run in terminal - takes input .csv and exports excel file

### Import Libraries that will be needed

In [1]:
import pandas as pd
import numpy as np
import datetime

### Import data from .csv

In [2]:
df_raw = pd.read_csv("maria_test.csv")

### Trim to 24 hour range

In [3]:
lower_limit = datetime.time(10,0,0)
upper_limit = datetime.time(10,5,0)
def get_data_range(L, lower_limit, upper_limit):
    """
    Find the start and end indices for a 24 hour range starting between a given time range
    Takes arguments L - a list of datetime.time objects, lower_limit and upper_limit - datetime.time objects 
    to specificy the range in which to choose the start time
    Returns a tuple: (start_index, end_index)
    """
    start_index = 0
    for i in range(len(L)):
        if L[i] >= lower_limit and L[i] < upper_limit:
            start_index = i
            break
    return (start_index, start_index + 288)


In [4]:
# Trim raw data to 24 hour window
df_raw.Time = pd.to_datetime(df_raw.Time, format='%H:%M:%S').dt.time
trim_index = get_data_range(list(df_raw.Time), lower_limit, upper_limit)
df = df_raw[trim_index[0]:trim_index[1]]
df = df.reset_index(drop=True)
trimmed_copy_df = df.copy()

### Make a column of bools to indicate day vs night

In [5]:
def isNight(x):
    """
    Take datetime.time object and return True if between 10pm and 10am, False otherwise 
    """
    night_start = datetime.time(22,0,0)
    night_end = datetime.time(10,0,0)
    if (x >= night_start and x < datetime.time(23, 59, 59)) or x < night_end:
        return True
    else:
        return False

In [6]:
# Create Boolean column indicating Night=True Day=False
# df.loc[:,'isNight'] = df.Time.apply(isNight) # If day/night are not split between top and bottom of window use this
df.isNight = pd.Series([x > len(df)/2-1 for x in range(len(df))]) # Faster / easier version accomplishes the same as above, IF day/night is split top/bottom


### Generate bout indices

In [7]:
# Script for collecting bouts
# Will need to create a list of bout indices to access resp data during bouts
def get_bout_indices(L):
    """
    Takes a list L and returns a list of tuples of the start/end indices in which sleeping bouts occurred.
    I.e. if two sleeping bouts occured, the first from index 5 to 20, and the second from index 30 to 40,
    this function will return [(5,20), (30,40)]
    """
    indices = []
    start_index = 1
    end_index = 1
    in_bout = False
    
    for i in range(len(L)):
        if L[i] == 0 and in_bout == False: 
            start_index = i
            in_bout = True
        if (L[i] != 0 or i == len(L)-1) and in_bout == True:
            end_index = i
            in_bout = False
            if i == len(L)-1:
                indices.append((start_index, end_index+1))
            else:
                indices.append((start_index, end_index))
    return indices 
        

# Better idea!!! Make a new column for each fly in the df to indicate True if sleep bout, False if wake.. This will make everything easier, and I already have the indices

In [8]:
ind = get_bout_indices(df.iloc[:,11])

In [9]:
def get_sleep_filter(indices):
    filter_list = []
    for t in indices:
        for i in range(t[0], t[1]):
            filter_list.append(i)
    return filter_list

In [10]:
x = get_sleep_filter(ind)

In [11]:
def create_sleeping_columns(df):
    for i in range(5):
        df.loc[:,'fly' + str(i+1) + 'sleeping'] = df.iloc[:,i+11].apply(lambda x: x == 0)
    

In [12]:
create_sleeping_columns(df)

In [13]:
df

,Time,Filename,mean_Deg_C_1_1,uL_BL,uL_Channel_2,uL_Channel_3,uL_Channel_4,uL_Channel_5,uL_Channel_6,Unnamed: 9,...,nsyb>GFP,nsyb>GFP.1,nsyb>mio,nsyb>mio.1,nsyb>mio.2,fly1sleeping,fly2sleeping,fly3sleeping,fly4sleeping,fly5sleeping
0,10:01:02,RepRec~07-02-2017_0268.exp,24.78638,0.230913,1.233220,1.716262,1.221084,1.099079,1.513888,NaN,...,0,4,0,1,0.0,True,False,True,False,True
1,10:06:02,RepRec~07-02-2017_0269.exp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,3,0,0,0.0,True,False,True,True,True
2,10:11:02,RepRec~07-02-2017_0270.exp,24.68581,0.207598,1.174517,1.607440,1.468174,1.131704,1.422653,NaN,...,0,0,49,30,17.0,True,True,False,False,False
3,10:16:02,RepRec~07-02-2017_0271.exp,24.63999,0.235605,1.475165,1.743097,1.525902,1.264942,1.763857,NaN,...,0,29,51,23,41.0,True,False,False,False,False
4,10:21:02,RepRec~07-02-2017_0272.exp,24.58665,0.271538,1.803171,2.192433,1.851901,1.589819,1.966495,NaN,...,9,37,63,69,81.0,False,False,False,False,False
5,10:26:02,RepRec~07-02-2017_0273.exp,24.56926,0.265697,1.895823,2.256947,1.827746,1.468000,2.020580,NaN,...,12,43,92,76,86.0,False,False,False,False,False
6,10:31:02,RepRec~07-02-2017_0274.exp,24.55996,0.276106,2.096711,2.278824,1.816047,1.457991,2.048677,NaN,...,1,47,90,66,90.0,False,False,False,False,False
7,10:36:02,RepRec~07-02-2017_0275.exp,24.56931,0.262771,2.132373,2.257428,1.716427,1.494124,2.072562,NaN,...,0,59,77,68,100.0,True,False,False,False,False
8,10:41:02,RepRec~07-02-2017_0276.exp,24.57437,0.264558,2.154877,2.271792,1.738834,1.509256,1.954420,NaN,...,0,68,79,82,87.0,True,False,False,False,False
9,10:46:02,RepRec~07-02-2017_0277.exp,24.60806,0.261818,2.165068,2.196250,1.735755,1.500451,1.834058,NaN,...,0,48,95,66,84.0,True,False,False,False,False


### Get day bouts and night bouts

In [14]:
# Need to put this into a function, and make a loop to generate day/night bouts for each fly of 5 in the df
colnames = df.columns[11:16]
def get_day_night_bouts(df, resp_colnum, sleep_colnum):
    """
    Takes a dataframe, the colnumn number for resp data, column number for sleep data
    Returns a tuple of two lists of lists: resp data for daytime sleep bouts, resp data for nighttime sleep bouts
    """
    bout_indices = get_bout_indices(list(df.iloc[:,sleep_colnum]))
    resp_bouts_day = []
    resp_bouts_night = []
    for i in bout_indices:
        if df.isNight[i[0]:i[1]].mean() >= 0.5:
            resp_bouts_night.append(list(df.iloc[:,resp_colnum])[i[0]:i[1]])
        else:
            resp_bouts_day.append(list(df.iloc[:,resp_colnum])[i[0]:i[1]])
    if not resp_bouts_day:
        resp_bouts_day = ['none']
    if not resp_bouts_night:
        resp_bouts_night = ['none']
    return resp_bouts_day, resp_bouts_night

def get_all_bouts(df, colnames):
    """
    """
    resp_colnum = 4
    sleep_colnum = 11
    colnames = df.columns[11:16]
    all_day_bouts = {x: [] for x in colnames}
    all_night_bouts = {x: [] for x in colnames}
    for i in range(5):
        day_bouts, night_bouts = get_day_night_bouts(df, resp_colnum, sleep_colnum)
        all_day_bouts[df.columns[i+11]] += day_bouts 
        all_night_bouts[df.columns[i+11]] += night_bouts 
        resp_colnum+=1
        sleep_colnum+=1
    return all_day_bouts, all_night_bouts

In [15]:
day_bouts_dict, night_bouts_dict = get_all_bouts(df, colnames)

In [16]:
def get_all_bouts_list(day_bouts_dict, night_bouts_dict, colnames):
    all_bouts_day_list = []
    all_bouts_night_list = []
    for i in colnames:
        all_bouts_day_list += day_bouts_dict[i] + [[]]
        all_bouts_night_list += night_bouts_dict[i] + [[]]
    return all_bouts_day_list, all_bouts_night_list

In [17]:
all_bouts_day_list, all_bouts_night_list = get_all_bouts_list(day_bouts_dict, night_bouts_dict, colnames)

In [18]:
# Create dataframes for day/night bouts
all_day_bouts_df = pd.DataFrame(all_bouts_day_list).transpose()
all_night_bouts_df = pd.DataFrame(all_bouts_night_list).transpose()

In [19]:
all_day_bouts_df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,1.233220,2.132373,2.178728,1.925485,2.141968,2.07424,2.112549,2.298214,2.092795,NaN,...,NaN,NaN,1.540798,1.4845,1.403534,NaN,1.513888,1.83716,1.814963,NaN
1,NaN,2.154877,2.265554,1.880769,2.112574,NaN,NaN,2.222638,2.102917,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.748227,NaN
2,1.174517,2.165068,NaN,NaN,2.183010,NaN,NaN,2.289908,2.087421,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.753722,NaN
3,1.475165,2.192669,NaN,NaN,NaN,NaN,NaN,2.147342,2.060248,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.747998,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.096391,2.043123,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.199318,1.992748,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.473719,2.208178,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.205356,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.223570,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.048538,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Get sleep profiles

In [20]:
def get_sleep_profile(df, resp_colnum, sleep_colnum):
    """
    Returns a list of tuples with sum of metabolic rate and the percentage sleep for each hour in the 24 hour window
    """
    fly1_sleep = df.iloc[:,sleep_colnum]
    fly1_resp = df.iloc[:,resp_colnum]
    hourly_resp_sleep = []
    for i in range(24):
        sleep = fly1_sleep[i*12:i*12+12]
        resp = fly1_resp[i*12:i*12+12]
        num_sleep_blocks = 0
        for i in sleep:
            if i == 0:
                num_sleep_blocks += 1
        sleep_avg = num_sleep_blocks / 12 * 100
        hourly_resp_sleep.append((resp.sum(), sleep_avg))
    return hourly_resp_sleep

def make_sleep_profile_dict(df):
    """
    Returns a dictionary containing a time index and the sleep profiles (both metabolic rate sum and average sleep)
    for each fly in the dataframe
    """
    resp_colnum = 4
    sleep_colnum = 11
    sleep_profile_dict = {}
    sleep_profile_dict['Time'] = [df.Time[x*12-1] for x in range(1, 25)]
    for i in range(5):
        sleep_profile = get_sleep_profile(df, resp_colnum, sleep_colnum)
        sleep_profile_dict[df.columns[sleep_colnum] + ' MR Sum'] = [x[0] for x in sleep_profile] #name of genotype is header of sleep column
        sleep_profile_dict[df.columns[sleep_colnum] + ' Avg Sleep'] = [x[1] for x in sleep_profile]
        resp_colnum+=1
        sleep_colnum+=1
    return sleep_profile_dict

def make_colnames(df):
    """
    Create list of ordered column names for dataframe to be created from sleep_profile dictionary
    """
    colnames = ['Time']
    for i in range(11,16):
        colnames.append(df.columns[i] + ' MR Sum')
        colnames.append(df.columns[i] + ' Avg Sleep')
    return colnames

In [21]:
#Creat test df for sleep profile
sleep_profile_dict = make_sleep_profile_dict(df)
sleep_profile_df = pd.DataFrame(sleep_profile_dict, columns=make_colnames(df))

In [22]:
sleep_profile_df

,Time,nsyb>GFP MR Sum,nsyb>GFP Avg Sleep,nsyb>GFP.1 MR Sum,nsyb>GFP.1 Avg Sleep,nsyb>mio MR Sum,nsyb>mio Avg Sleep,nsyb>mio.1 MR Sum,nsyb>mio.1 Avg Sleep,nsyb>mio.2 MR Sum,nsyb>mio.2 Avg Sleep
0,10:56:02,20.461218,66.666667,22.942187,8.333333,18.366464,16.666667,15.621530,8.333333,20.622934,16.666667
1,11:56:03,26.841970,16.666667,26.119280,0.000000,20.233120,0.000000,18.550534,0.000000,23.936785,0.000000
2,12:56:03,25.202784,16.666667,25.480481,0.000000,19.833503,0.000000,16.521188,0.000000,23.328866,0.000000
3,13:56:03,26.395106,41.666667,25.181367,0.000000,20.156252,41.666667,17.605225,0.000000,23.184750,0.000000
4,14:56:03,26.698553,8.333333,25.380688,0.000000,20.803472,0.000000,18.567816,8.333333,23.061054,8.333333
5,15:56:03,26.093465,91.666667,25.874774,33.333333,21.051349,0.000000,18.356043,0.000000,24.991610,0.000000
6,16:56:03,25.251128,100.000000,25.190239,41.666667,22.500076,91.666667,18.784424,8.333333,24.063960,8.333333
7,17:56:03,25.283425,100.000000,27.485837,16.666667,24.647229,16.666667,19.241208,8.333333,24.320807,25.000000
8,18:56:03,22.976131,100.000000,25.192950,0.000000,19.946764,0.000000,17.278564,0.000000,22.924898,0.000000
9,19:56:03,27.197133,100.000000,27.703722,0.000000,20.940079,0.000000,19.342446,0.000000,24.820388,0.000000


### Summary Stats
- ~~Minutes of Sleep~~
    - Total sleep
    - total day sleep,
    - total night sleep
- ~~Wake/Sleep MR~~
    - Mean Wake MR,
    - Mean Sleep MR,
    - Mean Day MR,
    - Mean Night MR,
    - Mean Day Sleep MR,
    - Mean Night Sleep MR~~
- Mean hourly slep
    - Mean hourly MR all
    - Mean hourly Day
    - Mean hourly MR night

In [23]:
#Total Sleep 
def get_sleep_minutes_df(df, colnames):
    total_sleep_list = [[x, ] for x in colnames]
    for i in range(len(colnames)):
        total_sleep_list[i].append(df.iloc[:,i+16].sum()*5)
        total_sleep_list[i].append(df.iloc[:,i+16][df.isNight == False].sum()*5)
        total_sleep_list[i].append(df.iloc[:,i+16][df.isNight == True].sum()*5)
    return pd.DataFrame(total_sleep_list, columns=['Fly', 'Total Sleep Min',
                                                   'Total Day Sleep Min', 'Total Night Sleep Min'])

In [24]:
sleep_minutes_df = get_sleep_minutes_df(df, colnames)

In [25]:
# Wake Sleep MR
def get_wake_sleep_mr_df(df, colnames):
    wake_sleep_mr_list = [[x, ] for x in colnames]
    for i in range(len(colnames)):
        wake_sleep_mr_list[i].append(df.iloc[:,i+4][df.iloc[:,i+16] == False].mean())
        wake_sleep_mr_list[i].append(df.iloc[:,i+4][df.iloc[:,i+16] == True].mean())
        wake_sleep_mr_list[i].append(df.iloc[:,i+4][df.isNight == False].mean())
        wake_sleep_mr_list[i].append(df.iloc[:,i+4][df.isNight == True].mean())
        wake_sleep_mr_list[i].append(df.iloc[:,i+4][df.iloc[:,i+16] == True][df.isNight == False].mean())
        wake_sleep_mr_list[i].append(df.iloc[:,i+4][df.iloc[:,i+16] == True][df.isNight == True].mean())
    return pd.DataFrame(wake_sleep_mr_list, columns=['Fly', 'Mean Wake MR', 'Mean Sleep MR', 'Mean Day MR', 
                                                     'Mean Night MR', 'Mean Day Sleep MR', 'Mean Night Sleep MR'])

In [26]:
wake_sleep_mr_df = get_wake_sleep_mr_df(df, colnames)

In [27]:
# Mean Hourly Sleep MR
def get_mean_hourly_sleep_mr_df(df, sleep_profile_df, colnames):
    mr_hourly = sleep_profile_df.iloc[:,1::2]
    mr_hourly.isNight = pd.Series([x > len(mr_hourly)/2-1 for x in range(len(mr_hourly))]) 
    mean_hourly_list = [[x, ] for x in colnames]
    for i in range(len(colnames)):
        mean_hourly_list[i].append(mr_hourly.iloc[:,i].mean())
        mean_hourly_list[i].append(mr_hourly.iloc[:,i][mr_hourly.isNight == False].mean())
        mean_hourly_list[i].append(mr_hourly.iloc[:,i][mr_hourly.isNight == True].mean())
    return pd.DataFrame(mean_hourly_list, columns=['Fly', 'Mean Hourly MR Total', 'Mean Hourly MR Day', 
                                                   'Mean Hourly MR Night'])

In [28]:
mean_hourly_sleep_mr_df = get_mean_hourly_sleep_mr_df(df, sleep_profile_df, colnames)

### To output to csv

In [29]:
trimmed_copy_df

,Time,Filename,mean_Deg_C_1_1,uL_BL,uL_Channel_2,uL_Channel_3,uL_Channel_4,uL_Channel_5,uL_Channel_6,Unnamed: 9,baseline,nsyb>GFP,nsyb>GFP.1,nsyb>mio,nsyb>mio.1,nsyb>mio.2
0,10:01:02,RepRec~07-02-2017_0268.exp,24.78638,0.230913,1.233220,1.716262,1.221084,1.099079,1.513888,NaN,0.0,0,4,0,1,0.0
1,10:06:02,RepRec~07-02-2017_0269.exp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,3,0,0,0.0
2,10:11:02,RepRec~07-02-2017_0270.exp,24.68581,0.207598,1.174517,1.607440,1.468174,1.131704,1.422653,NaN,0.0,0,0,49,30,17.0
3,10:16:02,RepRec~07-02-2017_0271.exp,24.63999,0.235605,1.475165,1.743097,1.525902,1.264942,1.763857,NaN,0.0,0,29,51,23,41.0
4,10:21:02,RepRec~07-02-2017_0272.exp,24.58665,0.271538,1.803171,2.192433,1.851901,1.589819,1.966495,NaN,0.0,9,37,63,69,81.0
5,10:26:02,RepRec~07-02-2017_0273.exp,24.56926,0.265697,1.895823,2.256947,1.827746,1.468000,2.020580,NaN,0.0,12,43,92,76,86.0
6,10:31:02,RepRec~07-02-2017_0274.exp,24.55996,0.276106,2.096711,2.278824,1.816047,1.457991,2.048677,NaN,0.0,1,47,90,66,90.0
7,10:36:02,RepRec~07-02-2017_0275.exp,24.56931,0.262771,2.132373,2.257428,1.716427,1.494124,2.072562,NaN,0.0,0,59,77,68,100.0
8,10:41:02,RepRec~07-02-2017_0276.exp,24.57437,0.264558,2.154877,2.271792,1.738834,1.509256,1.954420,NaN,0.0,0,68,79,82,87.0
9,10:46:02,RepRec~07-02-2017_0277.exp,24.60806,0.261818,2.165068,2.196250,1.735755,1.500451,1.834058,NaN,0.0,0,48,95,66,84.0


In [30]:
all_day_bouts_df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,1.233220,2.132373,2.178728,1.925485,2.141968,2.07424,2.112549,2.298214,2.092795,NaN,...,NaN,NaN,1.540798,1.4845,1.403534,NaN,1.513888,1.83716,1.814963,NaN
1,NaN,2.154877,2.265554,1.880769,2.112574,NaN,NaN,2.222638,2.102917,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.748227,NaN
2,1.174517,2.165068,NaN,NaN,2.183010,NaN,NaN,2.289908,2.087421,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.753722,NaN
3,1.475165,2.192669,NaN,NaN,NaN,NaN,NaN,2.147342,2.060248,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.747998,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.096391,2.043123,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.199318,1.992748,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.473719,2.208178,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.205356,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.223570,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.048538,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
all_night_bouts_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,2.104369,2.148405,2.098767,2.316298,2.166887,2.277380,2.129046,2.240794,NaN,2.107036,...,NaN,1.711097,1.872202,1.654127,1.604123,1.546667,1.493780,1.507254,1.310288,NaN
1,2.045786,2.094527,2.128810,NaN,2.148767,2.229456,2.081378,2.229203,NaN,2.011697,...,NaN,1.697962,1.697921,1.680200,1.606224,1.515770,1.481410,1.466357,1.320056,NaN
2,2.022999,NaN,2.125896,NaN,2.141979,2.104643,2.065192,2.248189,NaN,2.019161,...,NaN,1.662195,1.684184,1.628204,1.637692,1.482983,1.443135,1.420190,1.283597,NaN
3,2.004543,NaN,2.075255,NaN,2.121446,2.112358,2.068989,2.190412,NaN,2.002134,...,NaN,1.660565,1.672243,1.614598,1.587316,NaN,1.411354,1.417378,NaN,NaN
4,1.999814,NaN,2.022885,NaN,2.104481,2.115984,2.067090,2.152525,NaN,1.974879,...,NaN,1.644459,NaN,1.631852,1.627979,NaN,1.339271,1.365517,NaN,NaN
5,1.989032,NaN,2.057655,NaN,2.122109,2.106389,2.071055,2.078164,NaN,NaN,...,NaN,1.627199,NaN,1.655182,NaN,NaN,NaN,NaN,NaN,NaN
6,1.998919,NaN,2.020245,NaN,2.151854,2.131030,2.070767,2.046363,NaN,NaN,...,NaN,1.684558,NaN,1.632887,NaN,NaN,NaN,NaN,NaN,NaN
7,1.977594,NaN,2.021232,NaN,NaN,2.107472,2.068817,2.034193,NaN,NaN,...,NaN,1.630313,NaN,1.609408,NaN,NaN,NaN,NaN,NaN,NaN
8,2.044381,NaN,2.008784,NaN,NaN,NaN,2.090724,2.042698,NaN,NaN,...,NaN,1.663851,NaN,1.600656,NaN,NaN,NaN,NaN,NaN,NaN
9,2.164538,NaN,1.991202,NaN,NaN,NaN,2.064039,1.976686,NaN,NaN,...,NaN,1.672571,NaN,1.536134,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
sleep_profile_df

,Time,nsyb>GFP MR Sum,nsyb>GFP Avg Sleep,nsyb>GFP.1 MR Sum,nsyb>GFP.1 Avg Sleep,nsyb>mio MR Sum,nsyb>mio Avg Sleep,nsyb>mio.1 MR Sum,nsyb>mio.1 Avg Sleep,nsyb>mio.2 MR Sum,nsyb>mio.2 Avg Sleep
0,10:56:02,20.461218,66.666667,22.942187,8.333333,18.366464,16.666667,15.621530,8.333333,20.622934,16.666667
1,11:56:03,26.841970,16.666667,26.119280,0.000000,20.233120,0.000000,18.550534,0.000000,23.936785,0.000000
2,12:56:03,25.202784,16.666667,25.480481,0.000000,19.833503,0.000000,16.521188,0.000000,23.328866,0.000000
3,13:56:03,26.395106,41.666667,25.181367,0.000000,20.156252,41.666667,17.605225,0.000000,23.184750,0.000000
4,14:56:03,26.698553,8.333333,25.380688,0.000000,20.803472,0.000000,18.567816,8.333333,23.061054,8.333333
5,15:56:03,26.093465,91.666667,25.874774,33.333333,21.051349,0.000000,18.356043,0.000000,24.991610,0.000000
6,16:56:03,25.251128,100.000000,25.190239,41.666667,22.500076,91.666667,18.784424,8.333333,24.063960,8.333333
7,17:56:03,25.283425,100.000000,27.485837,16.666667,24.647229,16.666667,19.241208,8.333333,24.320807,25.000000
8,18:56:03,22.976131,100.000000,25.192950,0.000000,19.946764,0.000000,17.278564,0.000000,22.924898,0.000000
9,19:56:03,27.197133,100.000000,27.703722,0.000000,20.940079,0.000000,19.342446,0.000000,24.820388,0.000000


In [33]:
sleep_minutes_df

,Fly,Total Sleep Min,Total Day Sleep Min,Total Night Sleep Min
0,nsyb>GFP,1155,505,650
1,nsyb>GFP.1,615,60,555
2,nsyb>mio,825,140,685
3,nsyb>mio.1,620,20,600
4,nsyb>mio.2,565,35,530


In [34]:
wake_sleep_mr_df

,Fly,Mean Wake MR,Mean Sleep MR,Mean Day MR,Mean Night MR,Mean Day Sleep MR,Mean Night Sleep MR
0,nsyb>GFP,2.239575,2.094704,2.158882,2.088762,2.145303,2.056171
1,nsyb>GFP.1,2.205509,1.890080,2.208529,1.933102,2.039071,1.873973
2,nsyb>mio,1.786726,1.696233,1.792206,1.678261,1.856318,1.664683
3,nsyb>mio.1,1.496888,0.905025,1.552973,0.936032,1.476277,0.890744
4,nsyb>mio.2,2.011785,1.579045,2.037327,1.650022,1.735993,1.570161


In [35]:
mean_hourly_sleep_mr_df

,Fly,Mean Hourly MR Total,Mean Hourly MR Day,Mean Hourly MR Night
0,nsyb>GFP,25.305958,25.546772,25.065144
1,nsyb>GFP.1,24.665740,26.134255,23.197225
2,nsyb>mio,20.673448,21.207765,20.139131
3,nsyb>mio.1,14.804617,18.376848,11.232387
4,nsyb>mio.2,21.954318,24.108367,19.800270


## Export to .csv

In [36]:
# Now we save this list of lists as a csv
# resp_bouts_df = pd.DataFrame(resp_bouts).transpose()
# resp_bouts_df.to_csv('test.csv', index=False, header=False)